- loading data

In [33]:
! wget https://raw.githubusercontent.com/mohammedashrafdagga/ds_lr/main/dataset/melb_data.csv

--2022-01-24 15:50:36--  https://raw.githubusercontent.com/mohammedashrafdagga/ds_lr/main/dataset/melb_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2091239 (2.0M) [text/plain]
Saving to: ‘melb_data.csv.1’

melb_data.csv.1     100%[===================>]   1.99M  --.-KB/s    in 0.05s   

2022-01-24 15:50:36 (37.3 MB/s) - ‘melb_data.csv.1’ saved [2091239/2091239]



-- reading and importing libaray

In [34]:
# improt 
import pandas as pd

data_melb = pd.read_csv('melb_data.csv')

# looking a head of data
data_melb.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [35]:
data_melb.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

-- definte X and y

In [36]:
y = data_melb['Price']
X = data_melb.drop('Price', axis = 1)

In [37]:
X.head()

,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [38]:
# some prerocessing brefore start
object_col = [col for col in X.columns if X[col].dtype == 'object']
col_with_more_10_uinuqe_value = [col for col in object_col if len( X[col].unique()) > 10]
print(object_col)
print(col_with_more_10_uinuqe_value)

['Suburb', 'Address', 'Type', 'Method', 'SellerG', 'Date', 'CouncilArea', 'Regionname']
['Suburb', 'Address', 'SellerG', 'Date', 'CouncilArea']


In [39]:
X = X.drop(col_with_more_10_uinuqe_value, axis = 1)
X.columns

Index(['Rooms', 'Type', 'Method', 'Distance', 'Postcode', 'Bedroom2',
       'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [40]:
# split numerical and category data
num_col = list(X.select_dtypes(exclude='object').columns)
object_col = [col for col in X.columns if X[col].dtype == 'object']
print(object_col)
print(num_col)

['Type', 'Method', 'Regionname']
['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']


In [41]:
# import libaray used 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [42]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1, test_size = 0.2)

In [43]:
# apply in numberical
simputer = SimpleImputer(strategy='constant')

# apply in categroy
cat_transformar = Pipeline(steps=[
                                  ('imputer', SimpleImputer(strategy='most_frequent')),
                                  ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# columns transformar
preprocessor = ColumnTransformer(transformers=[
                                               ('num', simputer, num_col),
                                               ('cat', cat_transformar, object_col)
])

In [44]:
# create model 
model = RandomForestRegressor(n_estimators=100, random_state=1)

In [45]:
# apply preprocssor and model by pipeline
my_pipline = Pipeline(steps=[
                             ('preprocssor', preprocessor),
                             ('model', model)
])

In [46]:
# fitting model and predict
my_pipline.fit(X_train, y_train)

Pipeline(steps=[('preprocssor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Rooms', 'Distance',
                                                   'Postcode', 'Bedroom2',
                                                   'Bathroom', 'Car',
                                                   'Landsize', 'BuildingArea',
                                                   'YearBuilt', 'Lattitude',
                                                   'Longtitude',
                                                   'Propertycount']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('oneho

In [47]:
# predict model
pred = my_pipline.predict(X_test)

### Calcaluite MAE

In [48]:

print("MAE is :", mean_absolute_error(pred, y_test))

MAE is : 157178.0246242724
